# Generate a Modified Nested Set Index from NGA TDX-Hydro

This notebook demonstrates how to use functions in the [WikiWatershed/global-hydrography](https://github.com/WikiWatershed/global-hydrography) package to generate a modified nested set index using the TDX-Hydro datasets released by the [US National Geospatial-Intelligence Agency (NGA)](https://www.nga.mil).

This example notebook assumes that you have already downloaded the applicable data using the example provided in the `1_GetData.ipynb` notebook. This notebook also assumes that you will have completed the necessary setup steps outline in the **[Installation Instructions](README.md#get-started)** (and also completed as part of the notebook `1_GetData.ipynb`) 

The functions introduced in this notebook were developed in the `sandbox/modified_nested_set_index.ipynb` notebook.

# Python Imports

In this step we will import the necessary python dependencies for this example

In [1]:
from pathlib import Path
import re
from importlib import reload

import pyarrow as pa
import pyogrio
import geopandas as gpd
import pandas as pd

from global_hydrography.delineation.mnsi import modified_nest_set_index
from global_hydrography.preprocess import TDXPreprocessor

# Compile files that need to be processed

In this step we will compile a list of the files that need to be processed to have a modified nested set index. Note this step assumes that you have downloaded the files to the same directory and used the same naming convention as the `1_GetData.ipynb` example notebook. If you have opted to use a different location or naming convention you will need to modify this step accordingly.

In [2]:
# Confirm your current working directory (cwd) and repo/project directory
working_dir = Path.cwd()
project_dir = working_dir.parent
data_dir = project_dir / 'data_temp' # a temporary data directory that we .gitignore
tdx_dir = data_dir / 'nga'

In [3]:
#Scan the files in the data directory and only pull of the streamnet (blueline) files
files_to_process = []
for item in tdx_dir.iterdir():
    if item.is_file() and 'streamnet' in item.name and item.suffix=='.gpkg':
        files_to_process.append(item)

In [4]:
files_to_process

[PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_1020011530_01.gpkg'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg')]

# Compute the modified nested set index

In this step we will loop through each of the files to be processed, open them as a GeoDataFrame, applied the modified nested set algorithm, and then write them back to the original file. Note this steps assumes you have used the same file naming convention as the `1_GetData.ipynb` example notebook. If your naming convention is different, you may need to modify the code below. 

## `create_tdx_mnsi()` function

The following helper function was developed in the `sandbox/modified_nested_set_index.ipynb` notebook.

In [5]:
# define a helper function for the operation
def create_tdx_mnsi(file:Path, preprocessor:TDXPreprocessor) -> None:
    ''' Creates a Modified Nested Set Index from an original TDX-Hydro
    GeoPackage streamnet file, saving to a compressed GeoParquet file.

    The new GeoParquet file stores LINK numbers that are modified to be 
    globally unique, other useful orginal data fields, and three new MNSI
    fields. The GeoParquet file is saved with a filename in the form of:
    `f"{info['layer_name']}_mnsi.parquet"`.

    file: The Path to an original TDX-Hydro GeoPackage streamnet file.
    preprocessor: A TDXPreprocessor class instance. 

    Return: None
    '''

    # parse the file name to get the TDXHydroRegion
    tdx_hydro_region = int(re.search("\d{10}",file.name).group(0))
    print (f"Processing TDXHydroRegion = {tdx_hydro_region}")

    # get file metadata
    info = pyogrio.read_info(file, layer=0)
    print(f"  Reading: layer = {info['layer_name']} last updated {info['layer_metadata']['DBF_DATE_LAST_UPDATE']}")
    
    # open the file as GeoDataFrame
    gdf = gpd.read_file(file, engine='pyogrio', layer=0, use_arrow=True)

    # apply preprocessing to make linkno globally unique
    preprocessor.tdx_to_global_linkno(gdf, tdx_hydro_region)

    # apply preprocessing to make drop columns with no value
    preprocessor.tdx_drop_useless_columns(gdf)

    # compute the modified nested set index
    gdf = modified_nest_set_index(gdf)
    print('  Computed: modified nested set index')

    # Set 'LINKNO' as index, to facilitate selection
    gdf.set_index('LINKNO', inplace=True)

    # write back to the file
    tdx_parquet_path = tdx_dir / f"{info['layer_name']}_mnsi.parquet"
    gdf.to_parquet(tdx_parquet_path, compression='zstd')
    print(f'  File saved: {tdx_parquet_path.name}')

    return tdx_parquet_path

In [6]:
#initialize a preprocessor instance
#we want to reuse this object to take advantage of the cached TDX Basin Id crosswalk
preprocessor = TDXPreprocessor()

# Select file
file = files_to_process[1]

# Compute index
tdx_parquet_path = create_tdx_mnsi(file, preprocessor)

Processing TDXHydroRegion = 7020038340
  Reading: layer = TDX_streamnet_7020038340_01 last updated 2021-12-08
  Computed: modified nested set index
  File saved: TDX_streamnet_7020038340_01_mnsi.parquet


In [25]:
tdx_parquet_path.name

'TDX_streamnet_7020038340_01_mnsi.parquet'

In [7]:
# Get file size, in MB
tdx_parquet_path.stat().st_size / 1_000_000

186.781267

# Re-Read the Saved GeoParquet

In [8]:
# Open the file as GeoDataFrame
gdf = gpd.read_parquet(tdx_parquet_path)
gdf.info()
gdf

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 140097 entries, 750000000 to 750000589
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   DSLINKNO       140097 non-null  int32   
 1   USLINKNO1      140097 non-null  int32   
 2   USLINKNO2      140097 non-null  int32   
 3   ROOT_ID        140097 non-null  int32   
 4   DISCOVER_TIME  140097 non-null  int32   
 5   FINISH_TIME    140097 non-null  int32   
 6   strmOrder      140097 non-null  int32   
 7   Length         140097 non-null  float64 
 8   Magnitude      140097 non-null  int32   
 9   DSContArea     140097 non-null  float64 
 10  strmDrop       140097 non-null  float64 
 11  Slope          140097 non-null  float64 
 12  StraightL      140097 non-null  float64 
 13  USContArea     140097 non-null  float64 
 14  DOUTEND        140097 non-null  float64 
 15  DOUTSTART      140097 non-null  float64 
 16  DOUTMID        140097 non-null  float64 
 

,DSLINKNO,USLINKNO1,USLINKNO2,ROOT_ID,DISCOVER_TIME,FINISH_TIME,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,DOUTEND,DOUTSTART,DOUTMID,geometry
LINKNO,,,,,,,,,,,,,,,,,,
750000000,750001777,-1,-1,750021317,52,53,1,3847.9,1,9567845.0,42.07,0.010933,3233.7,5254867.5,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
750000001,750002369,-1,-1,750021317,49,50,1,2251.3,1,8768556.0,34.66,0.015397,1749.2,4320561.0,44802.7,47054.1,45928.4,"LINESTRING (-69.68589 46.40778, -69.686 46.407..."
750000593,750001777,-1,-1,750021317,51,52,1,1469.3,1,8466694.0,11.98,0.008153,1286.2,4319318.0,45853.6,47322.9,46588.3,"LINESTRING (-69.67822 46.41356, -69.67811 46.4..."
750001777,750002369,750000000,750000593,750021317,50,53,2,1050.9,2,19939082.0,0.91,0.000870,871.8,18034788.0,44802.7,45853.6,45328.2,"LINESTRING (-69.68589 46.40778, -69.68589 46.4..."
750000002,750004146,-1,-1,750021317,47,48,1,3551.0,1,9120895.0,67.48,0.019002,2593.6,5267176.0,41041.1,44591.7,42816.4,"LINESTRING (-69.687 46.37911, -69.687 46.379, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750000587,-1,-1,-1,750000587,1,2,1,2354.1,1,10233235.0,0.00,0.000000,1721.9,7569312.0,0.0,2354.1,1177.1,"LINESTRING (-81.59922 24.64033, -81.59911 24.6..."
750001180,-1,-1,-1,750001180,1,2,1,1326.7,1,9136435.0,0.00,0.000000,1072.3,4495984.5,0.0,1326.7,663.4,"LINESTRING (-81.63022 24.61767, -81.63011 24.6..."
750001772,-1,-1,-1,750001772,1,2,1,1000.1,1,4879280.0,0.00,0.000000,738.8,4387448.5,0.0,1000.1,500.0,"LINESTRING (-81.60144 24.58478, -81.60156 24.5..."


In [9]:
%%timeit
# Sorting index is fast if you want it.
gdf.sort_index()

23.2 ms ± 966 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
gdf.sort_index().info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 140097 entries, 750000000 to 750327711
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   DSLINKNO       140097 non-null  int32   
 1   USLINKNO1      140097 non-null  int32   
 2   USLINKNO2      140097 non-null  int32   
 3   ROOT_ID        140097 non-null  int32   
 4   DISCOVER_TIME  140097 non-null  int32   
 5   FINISH_TIME    140097 non-null  int32   
 6   strmOrder      140097 non-null  int32   
 7   Length         140097 non-null  float64 
 8   Magnitude      140097 non-null  int32   
 9   DSContArea     140097 non-null  float64 
 10  strmDrop       140097 non-null  float64 
 11  Slope          140097 non-null  float64 
 12  StraightL      140097 non-null  float64 
 13  USContArea     140097 non-null  float64 
 14  DOUTEND        140097 non-null  float64 
 15  DOUTSTART      140097 non-null  float64 
 16  DOUTMID        140097 non-null  float64 
 

## Read Only Non-Geometry Columns for Speed

This uses standard arguments in [`pandas.read_parquet()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html).

NOTE: It is necessary to use the Pandas method, because [`geopandas.read_parquet()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_parquet.html) will throw an error if no geometry field is read.

In [11]:
%%timeit
# Read entire file for comparison
gpd.read_parquet(tdx_parquet_path)
# 2.98 s ± 68 ms

3.04 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# Select all non-geometry fields
columns_to_read = list(gdf.columns)
columns_to_read.remove('geometry')

In [13]:
%%timeit
# Read non-geometry fields
# requires using Pandas method rather than GeoPandas
pd.read_parquet(tdx_parquet_path, columns=columns_to_read)
# 11.8 ms ± 481 µs

11.1 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Reading non-geometry fields gives a 250x speedup!!!**

In [14]:
# Save df for use in selecting rows, below
df = pd.read_parquet(tdx_parquet_path, columns=columns_to_read)

## Read Only Selected Rows for Speed

This uses `**kwargs` in [`geopandas.read_parquet()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_parquet.html) that are passed to [`pyarrow.parquet.read_table()`](https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_table.html#pyarrow.parquet.read_table).

Although partioning the GeoParquet can dramatically improve read performance, it is also possible to gain some benefit for non-partitioned files. See https://dzone.com/articles/parquet-data-filtering-with-pandas

In [15]:
%%timeit
# Read 4897 rows where 'ROOT_ID'==750288662
gpd.read_parquet(
    tdx_parquet_path,
    filters=[('ROOT_ID', '==', 750288662)]
)
# 1.57 s ± 35.7 ms

1.52 s ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
# Read 1 row where 'LINKNO'==750288662
gpd.read_parquet(
    tdx_parquet_path,
    filters=[('LINKNO', '==', 750288662)]
)
# 1.49 s ± 45.4 ms

1.47 s ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Reading selected rows gives up to a 2x speedup!!!**
- The limit appears to be constrained by the fact that pyarrow still needs to read the entire non-partioned

In [17]:
%%timeit
pa.parquet.read_table(
    tdx_parquet_path,
    # filters=[('LINKNO', '==', 750288662)]
)
# 1.44 s ± 7.06 ms

1.48 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
pa.parquet.read_table(
    tdx_parquet_path,
    filters=[('LINKNO', '==', 750288662)]
)
# 1.44 s ± 7.06 ms

1.47 s ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
pyogrio.read_arrow(tdx_parquet_path)
# 1.57 s ± 15.2 ms

1.68 s ± 89.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
pyogrio.read_info(tdx_parquet_path)

{'layer_name': 'TDX_streamnet_7020038340_01_mnsi',
 'crs': 'EPSG:4326',
 'encoding': 'UTF-8',
 'fields': array(['DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'ROOT_ID', 'DISCOVER_TIME',
        'FINISH_TIME', 'strmOrder', 'Length', 'Magnitude', 'DSContArea',
        'strmDrop', 'Slope', 'StraightL', 'USContArea', 'DOUTEND',
        'DOUTSTART', 'DOUTMID', 'LINKNO'], dtype=object),
 'dtypes': array(['int32', 'int32', 'int32', 'int32', 'int32', 'int32', 'int32',
        'float64', 'int32', 'float64', 'float64', 'float64', 'float64',
        'float64', 'float64', 'float64', 'float64', 'int32'], dtype=object),
 'fid_column': '',
 'geometry_name': 'geometry',
 'geometry_type': 'LineString',
 'features': 140097,
 'total_bounds': (-89.82122222222222,
  24.558999999998896,
  -66.14133333333214,
  46.44544444444445),
 'driver': 'Parquet',
 'capabilities': {'random_read': False,
  'fast_set_next_by_index': True,
  'fast_spatial_filter': False,
  'fast_feature_count': True,
  'fast_total_bounds': True},


# Explore Modified Nested Set

In [21]:
gdf[gdf.ROOT_ID==750288662].sort_values('DISCOVER_TIME')

,DSLINKNO,USLINKNO1,USLINKNO2,ROOT_ID,DISCOVER_TIME,FINISH_TIME,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,DOUTEND,DOUTSTART,DOUTMID,geometry
LINKNO,,,,,,,,,,,,,,,,,,
750288662,-1,750288070,750170262,750288662,1,4898,6,1273.1,2449,3.940295e+10,0.00,0.000000,1028.1,3.940148e+10,0.0,1273.1,636.6,"LINESTRING (-79.23711 33.13078, -79.23722 33.1..."
750170262,750288662,-1,-1,750288662,2,3,1,438.9,1,6.562896e+06,0.00,0.000000,431.8,4.331999e+06,1273.1,1712.0,1492.5,"LINESTRING (-79.24244 33.13889, -79.24244 33.1..."
750288070,750288662,750301685,750169670,750288662,3,4898,6,6269.9,2448,3.939491e+10,0.00,0.000000,5059.3,3.939078e+10,1273.1,7543.1,4408.1,"LINESTRING (-79.24244 33.13889, -79.24256 33.1..."
750169670,750288070,750169078,750168486,750288662,4,7,2,718.9,2,1.356416e+07,1.00,0.001386,595.9,1.348522e+07,7543.1,8262.0,7902.5,"LINESTRING (-79.28856 33.16289, -79.28856 33.1..."
750168486,750169670,-1,-1,750288662,5,6,1,336.1,1,4.427854e+06,0.04,0.000119,291.3,4.359848e+06,8262.0,8598.1,8430.1,"LINESTRING (-79.28778 33.15756, -79.28789 33.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750019844,750115156,750018660,750019252,750288662,4893,4898,2,2441.2,3,5.148408e+07,0.98,0.000400,1738.1,4.636870e+07,632110.8,634547.9,633329.4,"LINESTRING (-82.68056 35.00156, -82.68067 35.0..."
750019252,750019844,-1,-1,750288662,4894,4895,1,506.4,1,4.559639e+06,7.66,0.015123,482.9,4.342070e+06,634547.9,635053.5,634800.8,"LINESTRING (-82.699 34.99767, -82.69911 34.997..."
750018660,750019844,750017476,750018068,750288662,4895,4898,2,6750.6,2,4.180782e+07,24.27,0.003595,5175.3,1.942599e+07,634547.9,641287.2,637917.6,"LINESTRING (-82.699 34.99767, -82.69911 34.997..."
